## Notebook to generate mutation dataset for a single date

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime as dt

### JH Time Series Data

* Time series data is cumlative per column

In [2]:
date_up_to = '10/29/20'
jh_path = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/'
confirmed_global = 'time_series_covid19_confirmed_global.csv'
deaths_global = 'time_series_covid19_deaths_global.csv'

confirmed = pd.read_csv(jh_path+confirmed_global)
deaths = pd.read_csv(jh_path+deaths_global)

#Need to adjust country names to match those in CNCB
confirmed.loc[confirmed['Country/Region']=='US','Country/Region'] = 'United States'
confirmed.loc[confirmed['Country/Region']=='Congo (Kinshasa)','Country/Region'] = 'Democratic Republic of the Congo'
confirmed.loc[confirmed['Country/Region']=='Korea, South','Country/Region'] = 'South Korea'
confirmed.loc[confirmed['Country/Region']=='Czechia','Country/Region'] = 'Czech Republic'
confirmed.loc[confirmed['Country/Region']=='Burma','Country/Region'] = 'myanmar'
confirmed.loc[confirmed['Country/Region']=='Congo (Brazzaville)','Country/Region'] = 'republic of the congo'

confirmed['Country/Region'] = confirmed['Country/Region'].str.lower()

deaths.loc[deaths['Country/Region']=='US','Country/Region'] = 'United States'
deaths.loc[deaths['Country/Region']=='Congo (Kinshasa)','Country/Region'] = 'Democratic Republic of the Congo'
deaths.loc[deaths['Country/Region']=='Korea, South','Country/Region'] = 'South Korea'
deaths.loc[deaths['Country/Region']=='Czechia','Country/Region'] = 'Czech Republic'
deaths.loc[deaths['Country/Region']=='Burma','Country/Region'] = 'myanmar'
deaths.loc[deaths['Country/Region']=='Congo (Brazzaville)','Country/Region'] = 'republic of the congo'

deaths['Country/Region'] = deaths['Country/Region'].str.lower()

### Need other data table for population number
* doesn't matter for date as long as all the countries are covered, only getting population number

In [3]:
date = '10-29-2020.csv'
jh_path = '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
jh_data = pd.read_csv(jh_path+date)

#drop rows that have nans for incidence rate
jh_data = jh_data[~jh_data['Incidence_Rate'].isnull()]
jh_data = jh_data[jh_data['Incidence_Rate']!=0]

#matching countries of jh data to that of cncb countries
jh_data.loc[jh_data['Country_Region']=='Taiwan*','Country_Region'] = 'Taiwan'
jh_data.loc[jh_data['Country_Region']=='US','Country_Region'] = 'United States'
jh_data.loc[jh_data['Country_Region']=='Korea, South','Country_Region'] = 'South Korea'
jh_data.loc[jh_data['Country_Region']=='Czechia','Country_Region'] = 'Czech Republic'
jh_data.loc[jh_data['Country_Region']=='Burma','Country_Region'] = 'myanmar'
jh_data.loc[jh_data['Country_Region']=='Congo (Kinshasa)','Country_Region'] = 'Democratic Republic of the Congo'
jh_data.loc[jh_data['Country_Region']=='Congo (Brazzaville)','Country_Region'] = 'republic of the congo'

jh_data['Population'] = jh_data['Confirmed']/jh_data['Incidence_Rate']*100000
jh_data['Country_Region'] = jh_data['Country_Region'].str.lower()

### CNCB Genome Meta Data

In [4]:
#cncb meta, need to match countries
column_names = ['Virus Strain Name','Accession ID','Related ID','Nuc.Completeness','Sequence Quality','Host','Location','Sample Collection Date','Submitting Lab']
meta = pd.read_excel('../metadata10_27.xlsx',usecols=column_names,index_col=1)
meta.fillna(' ',inplace=True)
meta['Country'] = meta['Location'].str.split('/').str[0].str.strip()
meta.loc[meta['Country']=='\u200eRomania','Country'] = 'Romania'
#Remove low quality and partial reads, pretty sure cncb does not run variant annotation for these anyways
meta = meta[meta['Sequence Quality']!='Low']
meta = meta[meta['Nuc.Completeness']!='Partial']
meta['Country'] = meta['Country'].str.lower()

#Filter using date as well
meta = meta[meta['Sample Collection Date']!='2020-00-00'] #bad dates in there
meta.loc[:,'Sample Collection Date'] = pd.to_datetime(meta['Sample Collection Date'], yearfirst=True)
dt_date_up_to = dt.strptime(date_up_to, "%m/%d/%y")
meta = meta[meta.loc[:,'Sample Collection Date']<dt_date_up_to]

print(meta.shape)

(93012, 9)


### If there is a missing country in set difference, there will be nan data in output

In [5]:
set(meta['Country'])-set(confirmed['Country/Region'].str.lower())

set()

In [6]:
len(set(meta['Country']))

110

### Using Infections per 100k and Fatality percentage
* Currently summing over country labels, may be incorrect for some with territories, minimal impact

In [7]:
#Using coutries from meta, sum values to get counts per country
jh_country_lvl = pd.DataFrame(columns=['cases','deaths','population','Fatality_rate','Incidence_rate_per100k'])
for country in set(meta['Country'].str.lower()):
    summed_cases = confirmed.loc[confirmed['Country/Region']==country,date_up_to].sum()
    summed_deaths = deaths.loc[deaths['Country/Region']==country,date_up_to].sum()
    population = jh_data.loc[jh_data['Country_Region']==country,'Population'].sum()
    if summed_cases == 0:
        fate_rate = 0
    else:
        fate_rate = 100*summed_deaths/summed_cases
    inc_rate = 100000*summed_cases/population
    jh_country_lvl.loc[country] = [summed_cases,summed_deaths,population,fate_rate,inc_rate]
        
    
# jh_country_lvl['Fatality_rate'] = 100*jh_country_lvl['Deaths']/jh_country_lvl['Confirmed']
# jh_country_lvl['Incidence_rate_per100k'] = 100000*jh_country_lvl['Confirmed']/jh_country_lvl['Population']
# jh_country_lvl.replace([np.inf, -np.inf], 0,inplace=True)
print(jh_country_lvl.shape)
jh_country_lvl.sort_index(inplace=True)
jh_country_lvl

(110, 5)


,cases,deaths,population,Fatality_rate,Incidence_rate_per100k
algeria,57332.0,1949.0,43851043.0,3.399498,130.742614
andorra,4567.0,73.0,77265.0,1.598423,5910.826377
argentina,1143800.0,30442.0,45195777.0,2.661479,2530.767421
australia,27579.0,907.0,25459700.0,3.288734,108.324136
austria,93949.0,1056.0,9006400.0,1.124014,1043.135992
...,...,...,...,...,...
united states,8944934.0,228656.0,332804239.0,2.556263,2687.746414
uruguay,3044.0,57.0,3473727.0,1.872536,87.629224
venezuela,91280.0,789.0,28435943.0,0.864373,321.002191
vietnam,1177.0,35.0,97338583.0,2.973662,1.209181


### Parsing gff3 files
* Currently parse everytime you need to change the date

In [8]:
all_names = os.listdir('../gff3_cncb')
column_names = ['variant type','start','end', 'info']
missing = []
aggregated_mutations = {}

In [9]:
all_names = os.listdir('../gff3_cncb_restructured')
column_names = ['variant type','start','end', 'info']
missing = []
aggregated_mutations = {}

#for identifier in tqdm(meta[meta['Country']=='United States'].index):
for identifier in tqdm(meta.index):
    
    #Searching for correct identifier
    #--------------------------
    #No alternate name is ' '
    file_name = ''
    #Check if accession id in file names, if not check related ids
    if '2019-nCoV_'+identifier+'_variants.gff3' in all_names:
        file_name = '2019-nCoV_'+identifier+'_variants.gff3'
    # checking alternate names
    elif meta.loc[identifier,'Related ID'] != ' ':
        for alt_identifier in meta.loc[identifier,'Related ID'].replace(' ','').split(','):
            if '2019-nCoV_'+alt_identifier+'_variants.gff3' in all_names:
                file_name = '2019-nCoV_'+alt_identifier+'_variants.gff3'
                break
        #Added in case alternate names are also not found in gffs
        if file_name == '':
            missing.append(identifier)
            continue
    # If file name has not been updated, then there is no matching identifier, move to next index
    elif file_name == '':
        missing.append(identifier)
        continue
    #--------------------------
    
    #Filtering files with no variants
    #--------------------------
    with open(f'../gff3_cncb_restructured/{file_name}') as text_file:
        lines = text_file.readlines()
        counter = 0
        for l in lines:
            if '#' in l:
                counter += 1
    #Number of info lines should be less than total, if not then there are no mutations
    #--------------------------
    if counter<len(lines) and meta.loc[identifier,'Country'] in set(jh_data['Country_Region']): #country needs to be found in jh data for inf/fata rates
        
        gff = pd.read_csv(f'../gff3_cncb_restructured/{file_name}',sep='\t',skiprows=counter,usecols=[1,3,4,8],names=column_names)
        info_df = pd.DataFrame(gff['info'].str.split(';').values.tolist(),columns=[0,1,'Ref','Alt','Description']).drop([0,1],axis=1)
        gff = gff.drop(['info'],axis=1)
        gff['Country'] = [meta.loc[identifier,'Country']]*gff.shape[0]
        gff['infection_rate'] = [jh_country_lvl.loc[meta.loc[identifier,'Country'],'Incidence_rate_per100k']]*gff.shape[0]
        gff['fatality_rate'] = [jh_country_lvl.loc[meta.loc[identifier,'Country'],'Fatality_rate']]*gff.shape[0]
        temp_df = pd.concat([gff,info_df],axis=1)
        
        #Filtering alternate amino acid and reference for missense_variant and synonymous_variant
        missenses_ref = temp_df.loc[temp_df['Description'].str.contains('missense_variant'),'Description'].str.split(',').str[1].str[-3]
        synonymous_ref = temp_df.loc[temp_df['Description'].str.contains('synonymous_variant'),'Description'].str.split(',').str[1].str[-1]
        temp_df['Ref_AA'] = pd.concat([missenses_ref,synonymous_ref])
        temp_df['Alt_AA'] = temp_df.loc[temp_df['Description'].str.contains('missense_variant'),'Description'].str.split(',').str[1].str[-1]

        missenses_str = temp_df.loc[temp_df['Description'].str.contains('missense_variant'),'Description'].str.split(',').str[1].str.split('.').str[-1]
        synonymous_str = temp_df.loc[temp_df['Description'].str.contains('synonymous_variant'),'Description'].str.split(',').str[1].str.split('.').str[-1]
        temp_df['AA'] = pd.concat([missenses_str,synonymous_str])
        
        temp_df.fillna('', inplace=True)
        temp_df['descriptor'] = temp_df['start'].astype(str)+','+temp_df['end'].astype(str)+','+temp_df['Ref']+','+temp_df['Alt']+\
        ','+temp_df['Description'].str.split(',').str[0].str.split('=').str[1]+','+temp_df['variant type']+','+temp_df['Ref_AA']+','+temp_df['Alt_AA']+\
        ','+temp_df['AA']

#         if any(temp_df['Ref'].str.contains('REF=A') & temp_df['Alt'].str.contains('ALT=GGTTC')):
#             break
        
        for var in temp_df['descriptor']:
            if var not in aggregated_mutations.keys():
                aggregated_mutations[var] = [[],[],[]]
                
            #Below numbers should be the same for each mutation of same file
            #Country
            aggregated_mutations[var][0].append(temp_df.loc[0,'Country'])
            #Infection Rate
            aggregated_mutations[var][1].append(temp_df.loc[0,'infection_rate'])
            #Fatality Rate
            aggregated_mutations[var][2].append(temp_df.loc[0,'fatality_rate'])

print(len(missing))

100%|██████████| 67766/67766 [32:24<00:00, 34.85it/s]

7


In [10]:
unique_muts = pd.DataFrame(list(aggregated_mutations.keys()),columns=['descriptor'])
unique_muts = pd.DataFrame.join(unique_muts,pd.DataFrame(unique_muts['descriptor'].str.split(',').to_list())) #assign columns with parsed descriptor
unique_muts.set_index('descriptor',inplace=True)
unique_muts[0] = pd.to_numeric(unique_muts[0])
unique_muts[1] = pd.to_numeric(unique_muts[1])
unique_muts.sort_values([0,1],inplace=True)
unique_muts.columns = ['Start','End','Ref','Alt','VEP','Variant Type','Ref_AA','Alt_AA','AA']


counts = {}
num_countries = {}
infection_rate = {}
fatality_rate = {}
counted_countries = {}

for desc in unique_muts.index:
    counts[desc] = len(aggregated_mutations[desc][0])
    num_countries[desc] = len(set(aggregated_mutations[desc][0]))
    infection_rate[desc] = np.mean(aggregated_mutations[desc][1])
    fatality_rate[desc] = np.mean(aggregated_mutations[desc][2])
    counted_countries[desc] = dict(Counter(aggregated_mutations[desc][0]))
        
unique_muts['counts'] = unique_muts.index.to_series().map(counts)
unique_muts['countries'] = unique_muts.index.to_series().map(num_countries)
unique_muts['infection_rate'] = unique_muts.index.to_series().map(infection_rate)
unique_muts['fatality_rate'] = unique_muts.index.to_series().map(fatality_rate)
unique_muts['counted_countries'] = unique_muts.index.to_series().map(counted_countries)
unique_muts.index = unique_muts.index.str.split(',').str[0:4].str.join('_')
unique_muts.to_csv(f"temporal_files/added_b/unique_mutations_country{'_'.join(date_up_to.split('/'))}.csv")

In [11]:
unique_muts

,Start,End,Ref,Alt,VEP,Variant Type,Ref_AA,Alt_AA,AA,counts,countries,infection_rate,fatality_rate,counted_countries
descriptor,,,,,,,,,,,,,,
1_1_REF=AA_ALT=-,1,1,REF=AA,ALT=-,intergenic_variant,Deletion,,,,966,33,815.093885,4.996641,"{'china': 15, 'singapore': 8, 'france': 2, 'ca..."
1_1_REF=A_ALT=CCAGAAGAA,1,1,REF=A,ALT=CCAGAAGAA,intergenic_variant,Indel,,,,2,1,6.136787,5.394306,{'china': 2}
1_1_REF=A_ALT=G,1,1,REF=A,ALT=G,intergenic_variant,SNP,,,,70,10,1064.553193,3.886944,"{'china': 10, 'united states': 42, 'chile': 7,..."
1_1_REF=A_ALT=CCCTGAATTAGACTCA,1,1,REF=A,ALT=CCCTGAATTAGACTCA,intergenic_variant,Indel,,,,1,1,6.136787,5.394306,{'china': 1}
1_1_REF=A_ALT=GGTTC,1,1,REF=A,ALT=GGTTC,intergenic_variant||intergenic_variant,Indel,,,,1,1,6.136787,5.394306,{'china': 1}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29903_29903_REF=-_ALT=-ANANANAC,29903,29903,REF=-,ALT=-ANANANAC,intergenic_variant,Insertion,,,,1,1,1235.078018,3.558906,{'united states': 1}
29903_29903_REF=-_ALT=-AAANANANANAAAAAAA,29903,29903,REF=-,ALT=-AAANANANANAAAAAAA,intergenic_variant,Insertion,,,,1,1,1235.078018,3.558906,{'united states': 1}
29903_29903_REF=A_ALT=ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,29903,29903,REF=A,ALT=ACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,intergenic_variant,Insertion,,,,1,1,580.396386,10.436787,{'spain': 1}


In [12]:
unique_muts[unique_muts['VEP']=='missense_variant']

,Start,End,Ref,Alt,VEP,Variant Type,Ref_AA,Alt_AA,AA,counts,countries,infection_rate,fatality_rate,counted_countries
descriptor,,,,,,,,,,,,,,
269_269_REF=G_ALT=A,269,269,REF=G,ALT=A,missense_variant,SNP,E,K,2E>K,1,1,447.212807,13.747245,{'united kingdom': 1}
270_270_REF=A_ALT=G,270,270,REF=A,ALT=G,missense_variant,SNP,E,G,2E>G,1,1,6.136787,5.394306,{'china': 1}
271_271_REF=G_ALT=T,271,271,REF=G,ALT=T,missense_variant,SNP,E,D,2E>D,3,1,447.212807,13.747245,{'united kingdom': 3}
273_273_REF=G_ALT=A,273,273,REF=G,ALT=A,missense_variant,SNP,S,N,3S>N,1,1,1235.078018,3.558906,{'united states': 1}
274_274_REF=C_ALT=A,274,274,REF=C,ALT=A,missense_variant,SNP,S,R,3S>R,2,1,1235.078018,3.558906,{'united states': 2}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29666_29666_REF=C_ALT=A,29666,29666,REF=C,ALT=A,missense_variant,SNP,L,I,37L>I,1,1,447.212807,13.747245,{'united kingdom': 1}
29666_29666_REF=C_ALT=G,29666,29666,REF=C,ALT=G,missense_variant,SNP,L,V,37L>V,1,1,447.212807,13.747245,{'united kingdom': 1}
29667_29667_REF=T_ALT=C,29667,29667,REF=T,ALT=C,missense_variant,SNP,L,P,37L>P,1,1,1235.078018,3.558906,{'united states': 1}
